# Introduction  
## Nonlinear Regression
- Suppose we have the following regression problem:  
<div align="center"><img src = "./nonlinear.jpg" width = '500' height = '100' align = center /></div>  
- What are some options?   
- basis functions, kernel methods, trees, neural nets, ...  

## Linear Model with Basis Functions
- Choose some basis functions on input space $X$  
$$
g_{1}, \ldots, g_{M}: X \rightarrow \mathrm{R}
$$  
- Predict with linear combination of basis functions:  
$$
f(x)=\sum_{m=1}^{M} v_{m} g_{m}(x)
$$  
- Can ﬁt this using standard methods for linear models (e.g. least squares, lasso, ridge, etc.)
- In ML parlance, basis functions are called features or feature functions.
- $f (x)$ is a number — for regression, it’s exactly what we’re looking for.   
- Otherwise, $f (x)$ is often called a score function  
- It can be   
  - thresholded to get a classiﬁcation  
  - transformed to get a probability   
  - transformed to get a parameter of a probability distribution (e.g. Poisson regression)  
  - used for ranking search results

## Adaptive Basis Function Model  
- Base hypothesis space $H$ consisting of functions $h : X \to R$.  
  - We will choose our “basis functions” or “features” from this set of functions  
- An adaptive basis function expansion over $H$ is  
$$
f(x)=\sum_{m=1}^{M} v_{m} h_{m}(x)
$$  
where $v_m \in R$ and $h_m \in H$ are chosen based on training data.

- Combined hypothesis space: $F_M$:  
$$
\mathcal{F}_{M}=\left\{\sum_{m=1}^{M} v_{m} h_{m}(x) \mid v_{m} \in \mathbf{R}, h_{m} \in \mathcal{H}, m=1, \ldots, M\right\}
$$  
- Suppose we’re given some data $D = ((x_1,y_1),...,(x_n,y_n))$.  
- Learning is choosing $v_1,...,v_M \in R$ and $h_1,...,h_M \in H$ to ﬁt $D$.  

## Empirical Risk Minimization
- We’ll consider learning by **empirical risk minimization**:  
$$
\hat{f}=\underset{f \in \mathcal{F}_{M}}{\operatorname{argmin}} \frac{1}{n} \sum_{i=1}^{n} \ell\left(y_{i}, f\left(x_{i}\right)\right)
$$  
for some function $l(\hat(y),y)$  
- Write ERM objective function as  
$$
J\left(v_{1}, \ldots, v_{M}, h_{1}, \ldots, h_{M}\right)=\frac{1}{n} \sum_{i=1}^{n} \ell\left(y_{i}, \sum_{m=1}^{M} v_{m} h_{m}(x)\right)
$$  
- How to optimize J? i.e. how to learn?

## Gradient-Based Methods
- Suppose our base hypothesis space is parameterized by $\Theta =R^b$:(Linear Base space)  
$$
J\left(v_{1}, \ldots, v_{M}, \theta_{1}, \ldots, \theta_{M}\right)=\frac{1}{n} \sum_{i=1}^{n} \ell\left(y_{i}, \sum_{m=1}^{M} v_{m} h\left(x ; \theta_{m}\right)\right)
$$  
- Can we can diﬀerentiate $J$ w.r.t. $v_m$’s and $\theta_m$’s? Optimize with SGD?  
- For some hypothesis spaces and typical loss functions, yes!   
- Neural networks fall into this category! ($h_1,...,h_M$ are neurons of last hidden layer.)




  

## What if Gradient Based Methods Don’t Apply?  
- What if base hypothesis space $H$ consists of decision trees?  
- Can we even parameterize trees with $\Theta = R^b$?  
- Even if we could for some set of trees,   
  - predictions would not change continuously w.r.t. $\theta \in \Theta$,  
  - and so certainly not diﬀerentiable  
- Today we’ll discuss **gradient boosting**. It applies whenever   
  - our loss function is [sub]diﬀerentiable w.r.t. training predictions $f (x_i)$, and   
  - we can do regression with the base hypothesis space $H $(e.g. regression trees).  

## Overview  
- Forward stagewise additive modeling (FSAM)   
  - example: L2 Boosting 
  - example: exponential loss gives AdaBoost 
  - Not clear how to do it with many other losses, including logistic loss   
- Gradient Boosting example:   
  - logistic loss gives BinomialBoost   
- Variations on Gradient Boosting   
  - step size selection   
  - stochastic row/column selection   
  - XGBoost  
  

# Forward stagewise additive modeling (FSAM)    
- FSAM is an iterative optimization algorithm for ﬁtting adaptive basis function models.  
- Start with $f_0 ≡ 0$.  
- After $m−1$ stages, we have  
$$
f_{m-1}=\sum_{i=1}^{m-1} v_{i} h_{i}
$$  
- In $m$’th round, we want to ﬁnd   
  - step direction $h_m \in H$ (i.e. a basis function) and 
  - step size $v_i > 0$  
- such that
$$
f_{m}=f_{m-1}+v_{i} h_{m}
$$  
improves objective function value by as much as possible  

## Forward Stagewise Additive Modeling for ERM
- Initialize $f_0(x) =0$.  
-  For $m = 1$ to $M$:  
  -  Compute:
$$
\left(v_{m}, h_{m}\right)=\underset{v \in \mathbf{R}, h \in \mathcal{H}}{\arg \min } \frac{1}{n} \sum_{i=1}^{n} \ell(y_{i}, f_{m-1}\left(x_{i}\right) \underbrace{+v h\left(x_{i}\right)}_{\text {new piece }})
$$  
  - Set $f_m = f_{m−1} +ν_m h$.  
-  Return: $f_M$.  



# Example $L^2$ Boosting  
- Suppose we use the square loss. Then in each step we minimize  
$$
J(v, h)=\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-[f_{m-1}\left(x_{i}\right) \underbrace{+v h\left(x_{i}\right)}_{\text {new piece }}]\right)^{2}
$$  
- If $H$ is closed under rescaling (i.e. if $h\in H$, then $vh \in H$ for all $h \in R$),  then don’t need $ν$.  
- Take $ν = 1$ and minimize   
$$
J(h)=\frac{1}{n} \sum_{i=1}^{n}\left(\left[y_{i}-f_{m-1}\left(x_{i}\right)\right]-h\left(x_{i}\right)\right)^{2}
$$  
- This is just ﬁtting the residuals with least-squares regression!   
- If we can do regression with our base hypothesis space $H$, then we’re set!  

## Recall: Regression Stumps
- A regression stump is a function of the form $h(x) = a1(x_i \leq c)+b1(x_i > c)$
<div align="center"><img src = "./regression stump.jpg" width = '500' height = '100' align = center /></div>   

## L2 Boosting with Decision Stumps: Demo
- Consider FSAM with $L_2$ loss (i.e. $L_2$ Boosting)  
- For base hypothesis space of regression stumps  
- Data we’ll ﬁt with code:
<div align="center"><img src = "./data.jpg" width = '500' height = '100' align = center /></div>   
  
## L2 Boosting with Decision Stumps: Results
<div align="center"><img src = "./result.jpg" width = '500' height = '100' align = center /></div>   

<div align="center"><img src = "./result2.jpg" width = '500' height = '100' align = center /></div>  

<div align="center"><img src = "./result3.jpg" width = '500' height = '100' align = center /></div>  

# Example: AdaBoost  
## The Classiﬁcation Problem
- Outcome space $Y = \{−1,1\}$  
- Action space $A = R$  
- Score function $f : X \to A$.  
- Margin for example $(x,y)$ is $m = yf (x)$.  
  - $m > 0 \Leftrightarrow$ classiﬁcation correct   
  - Larger $m$ is better.  
  
## Margin-Based Losses for Classiﬁcation
- Introduce the exponential loss: $l(y,f (x)) = e^{−yf (x)}$.
<div align="center"><img src = "./exponential loss.jpg" width = '500' height = '100' align = center /></div>   

## FSAM with Exponential Loss
- Consider classiﬁcation setting: $Y = \{−1,1\}$.  
- Take loss function to be the **exponential loss**: $l(y,f (x)) = e^{−yf (x)}$.  
- Let $H$ be a base hypothesis space of classiﬁers $h : X \to \{−1,1\}$.  
- Then Forward Stagewise Additive Modeling (FSAM) reduces to a **version of AdaBoost**.    
**Note that exponential loss puts a very large weight on bad misclassiﬁcations**  

## AdaBoost / Exponential Loss: Robustness Issues
- When Bayes error rate is high (e.g. $P(f^∗(X)\ne Y) = 0.25)$  
  - e.g. there’s some intrinsic randomness in the label   
  - e.g. training examples with same input, but diﬀerent classiﬁcations.   
- Best we can do is predict the most likely class for each $x$.   
- Some training predictions should be wrong,  
  - because example doesn’t have majority class  
  - AdaBoost / exponential loss puts a lot of focus on getting those right  
- Empirically, AdaBoost has degraded performance in situations with  
  - high Bayes error rate, or when there’s high “label noise”   
- Logistic loss performs better in settings with high Bayes error

## FSAM for Other Loss Functions
- We know how to do FSAM for certain loss functions
  - e.g square loss, absolute loss, exponential loss,...   
- In each case, happens to reduce to another problem we know how to solve.   
- However, not clear how to do FSAM in general  
- For example, logistic loss / cross-entropy loss?  



# Gradient Boost/ Any Boost  
## FSAM Is Iterative Optimization  
- The FSAM step
$$
\left(v_{m}, h_{m}\right)=\underset{v \in \mathbf{R}, h \in \mathcal{H}}{\arg \min } \frac{1}{n} \sum_{i=1}^{n} \ell(y_{i}, f_{m-1}\left(x_{i}\right) \underbrace{+v h\left(x_{i}\right)}_{\text {new piece }})
$$  
- Hard part: ﬁnding the best step direction $h$.   
- What if we looked for the locally best step direction?  
  - like in gradient descent

## “Functional” Gradient Descent
- We want to minimize
$$
J(f)=\sum_{i=1}^{n} \ell\left(y_{i}, f\left(x_{i}\right)\right)
$$  
- In some sense, we want to take the gradient w.r.t. $f$, whatever that means.  
- $J(f)$ only depends on $f$ at the n training points.  
- Define  
$$
\mathbf{f}=\left(f\left(x_{1}\right), \ldots, f\left(x_{n}\right)\right)^{T}
$$  
- and write the objective function as
$$
J(\mathbf{f})=\sum_{i=1}^{n} \ell\left(y_{i}, \mathbf{f}_{i}\right)
$$  

## Functional Gradient Descent: Unconstrained Step Direction
- Consider gradient descent on
$$
J(\mathbf{f})=\sum_{i=1}^{n} \ell\left(y_{i}, \mathbf{f}_{i}\right)
$$  
- The **negative gradient step direction** at $f$ is  
$$
\begin{aligned}
-\mathbf{g} &=-\nabla_{\boldsymbol{f}} J(\mathbf{f}) \\
&=-\left(\partial_{\mathbf{f}_{1}} \ell\left(y_{1}, \mathbf{f}_{1}\right), \ldots, \partial_{\mathbf{f}_{n}} \ell\left(y_{n}, \mathbf{f}_{n}\right)\right)
\end{aligned}
$$  
which we can easily calculate.   
- $-g\in R_n$ is the direction we want to change each of our $n$ predictions on training data.   
- Eventually we need more than just $\mathbf{f}$, which is just predictions on training.

## Unconstrained Functional Gradient Stepping

<div align="center"><img src = "./functional boost.jpg" width = '500' height = '100' align = center /></div>   

- $R(f)$ is the empirical risk, where $f = (f (x_1),f (x_2))$ are predictions on training set.   
- Issue: $\hat{f}_M$ only deﬁned at training points  

## Functional Gradient Descent: Projection Step
- Unconstrained step direction is  
$$
-\mathbf{g}=-\nabla_{\boldsymbol{f}} J(\mathbf{f})=-\left(\partial_{\mathbf{f}_{1}} \ell\left(y_{1}, \mathbf{f}_{1}\right), \ldots, \partial_{\mathbf{f}_{n}} \ell\left(y_{n}, \mathbf{f}_{n}\right)\right)
$$   
- Also called the “pseudo-residuals”   
  - (for square loss, they’re exactly the residuals)   
- Find the closest base hypothesis $h \in H$ in $l_2$ sense  
$$
\min _{h \in \mathcal{H}} \sum_{i=1}^{n}\left(-\mathbf{g}_{i}-h\left(x_{i}\right)\right)^{2}
$$  
- This is a least squares regression problem over hypothesis space $H$.  
- **Take the $h\in H$ that best approximates $−g$ as our step direction.**  


  

<div align="center"><img src = "./projection.jpg" width = '500' height = '100' align = center /></div>   
$T(x;p)\in H$ is our actual step direction – like the projection of $-g=-\nabla R(f)$ onto $H$.  

## Functional Gradient Descent: Step Size
- Finally, we choose a stepsize.  
- Option 1 (Line search):
$$
v_{m}=\underset{v>0}{\arg \min } \sum_{i=1}^{n} \ell\left\{y_{i}, f_{m-1}\left(x_{i}\right)+v h_{m}\left(x_{i}\right)\right\}
$$    
- Option 2: (Shrinkage parameter – more common)   
  - We consider $ν =1$ to be the full gradient step.   
  - Choose a ﬁxed $ν\in (0,1)$ – called a shrinkage parameter  
  - A value of $ν =0.1$ is typical – optimize as a hyperparameter .

## The Gradient Boosting Machine Ingredients (Recap)
- Take any [sub]diﬀerentiable loss function.   
- Choose a base hypothesis space for regression.  
- Choose number of steps (or a stopping criterion).   
- Choose step size methodology.   
- Then you’re good to go!



# Example: BinomialBoost  
## BinomialBoost: Gradient Boosting with Logistic Loss
- Recall the logistic loss for classiﬁcation, with $Y = \{−1,1\}$:  
$$
\ell(y, f(x))=\log \left(1+e^{-y f(x)}\right)
$$  
- Pseudoresidual for $i$’th example is negative derivative of loss w.r.t. prediction:   
$$
\begin{aligned}
r_{i} &=-\partial_{f\left(x_{i}\right)}\left[\log \left(1+e^{-y_{i} f\left(x_{i}\right)}\right)\right] \\
&=\frac{y_{i} e^{-y_{i} f\left(x_{i}\right)}}{1+e^{-y_{i} f\left(x_{i}\right)}} \\
&=\frac{y_{i}}{1+e^{y_{i} f\left(x_{i}\right)}}
\end{aligned}
$$   
- Pseudoresidual for $i$th example:   
$$
r_{i}=-\partial_{f\left(x_{i}\right)}\left[\log \left(1+e^{-y_{i} f\left(x_{i}\right)}\right)\right]=\frac{y_{i}}{1+e^{y_{i} f\left(x_{i}\right)}}
$$  
So if $f_{m−1}(x)$ is prediction after $m−1$ rounds, step direction for $m$’th round is  
$$
h_{m}=\underset{h \in \mathcal{H}}{\operatorname{argmin}} \sum_{i=1}^{n}\left[\left(\frac{y_{i}}{1+e^{y_{i} f_{m-1}\left(x_{i}\right)}}\right)-h\left(x_{i}\right)\right]^{2}
$$  
- and
$$
f_{m}(x)=f_{m-1}(x)+v h_{m}(x)
$$


# Gradient Tree Boosting  
## Gradient Tree Boosting
- One common form of gradient boosting machine takes
$$
H = \{\text{regression trees of size } J\},
$$  
where $J$ is the number of terminal nodes.  
- $J = 2$ gives decision stumps  
- HTF recommends $4\leq J \leq 8$ (but more recent results use much larger trees)   
- Software packages:   
  - Gradient tree boosting is implemented by the **gbm package for R**   
  - as GradientBoostingClassifier and GradientBoostingRegressor in sklearn  
  - **xgboost** and **lightGBM** are state of the art for speed and performance  
  
# GBM Regression with Stumps  
Sinc Function: Our Dataset
<div align="center"><img src = "./sinc.jpg" width = '500' height = '100' align = center /></div>   

## Minimizing Square Loss with Ensemble of Decision Stumps
<div align="center"><img src = "./ensemble stumps.jpg" width = '500' height = '100' align = center /></div>   

- Decision stumps with $1,10,50,$ and $100$ steps, step size $\lambda = 1$.  

## Step Size as Regularization
<div align="center"><img src = "./lambda.jpg" width = '500' height = '100' align = center /></div>   

## Rule of Thumb
- The smaller the step size, the more steps you’ll need.  
- But never seems to make results worse, and often better.  
- So set your step size as small as you have patience for.

# Variation on Gradient Boosting  
## Stochastic Gradient Boosting
- For each stage,   
  - choose random subset of data for computing projected gradient step.  
  - “Typically, about 50% of the dataset size, can be much smaller for large training set.”   
  - Fraction is called the bag fraction.   
- Why do this?   
  - Subsample percentage is additional regularization parameter – may help overﬁtting.   
  - Faster.   
- We can view this is a **minibatch method**. 
  - we’re estimating the “true” step direction (the projected gradient) using a subset of data

## Bag as Minibatch
- Just as we argued for minibatch SGD,   
  - sample size needed for a good estimate of step direction is independent of training set size   
- Minibatch size should depend on   
  - the complexity of base hypothesis space   
  - the complexity of the target function (Bayes decision function)   
- Seems like an interesting area for both practical and theoretical pursuit.
- Column / Feature Subsampling for Regularization  
  - Similar to random forest, randomly choose a subset of features for each round.
  - XGBoost paper says: “According to user feedback, using column sub-sampling prevents overﬁtting even more so than the traditional row sub-sampling.”

## Newton Step Direction
- For GBM, we ﬁnd the closest $h\in F$ to the negative gradient   
$$
-\mathbf{g}=-\nabla_{\boldsymbol{f}} J(\mathbf{f})
$$  
- This is a “ﬁrst order” method.  
- Newton’s method is a “second order method”:  
  - Find 2nd order (quadratic) approximation to $J$ at $f$.  
  - Requires computing gradient and Hessian of $J$.  
  - Newton step direction points towards minimizer of the quadratic  
  - Minimizer of quadratic is easy to ﬁnd in closed form  
- Boosting methods with projected Newton step direction:   
  - LogitBoost (logistic loss function)   
  - XGBoost (any loss – uses regression trees for base classiﬁer)



## Newton Step Direction for GBM
- Generically, second order Taylor expansion of $J$ at $f$ in direction $r$  
$$
J(\mathbf{f}+\mathbf{r})=J(\mathbf{f})+\left[\nabla_{\mathbf{f}} J(\mathbf{f})\right]^{T} \mathbf{r}+\frac{1}{2} \mathbf{r}^{T}\left[\nabla_{\mathbf{f}}^{2} J(\mathbf{f})\right] \mathbf{r}
$$  
- For $J(\mathbf{f})=\sum_{i=1}^{n} \ell\left(y_{i}, \mathbf{f}_{i}\right)$  
$$
J(\mathbf{f}+\mathbf{r})=\sum_{i=1}^{n}\left[\ell\left(y_{i}, \mathbf{f}_{i}\right)+g_{i} \mathbf{r}_{i}+\frac{1}{2} h_{i} \mathbf{r}_{i}^{2}\right]
$$  
where $g_{i}=\partial_{\mathbf{f}_{i}} \ell\left(y_{i}, \mathbf{f}_{i}\right)$, and $h_{i}=\partial_{\mathbf{f}_{i}}^{2} \ell\left(y_{i}, \mathbf{f}_{i}\right)$  
- Can ﬁnd $r$ that minimizes $J(f+r)$ in closed form  
- Can take step direction to be “projection” of $r$ into base hypothesis space $H$.  

## XGBoost: Objective Function with Tree Penalty Term
- Adds explicit penalty term on tree complexity to the empirical risk:  
$$
\Omega(r)=\gamma T+\frac{1}{2} \lambda \sum_{i=1}^{T} w_{j}^{2}
$$  
where $r \in H$ is a regression tree from our base hypothesis space and   
  - $T$ is the number of leaf nodes and   
  - $w_j$ is the prediction in the $j$’th node   
- Objective function at step $m$:  
$$
J(r)=\sum_{i=1}^{n}\left[g_{i} r\left(x_{i}\right)+\frac{1}{2} h_{i} r\left(x_{i}\right)^{2}\right]+\Omega(r)
$$  
- In XGBoost, they also use this objective to decide on tree splits  

## XGBoost: Rewriting objective function
- For a given tree, let $q(x_i)$ be $x_i$’s node assignment and $w_j$ the prediction for node $j$.  
- In each step of XGBoost we’re looking for a tree that minimizes  
$$
\sum_{i=1}^{n}\left[g_{i} w_{q\left(x_{i}\right)}+\frac{1}{2} h_{i} w_{q\left(x_{i}\right)}^{2}\right]+\gamma T+\frac{1}{2} \lambda \sum_{i=1}^{T} w_{j}^{2}
$$
$$
=\sum_{\text {leaf node } j=1}^{T}\left[(\underbrace{\sum_{i \in I_{j}} g_{i}}_{G_{j}}) w_{j}+\frac{1}{2}(\underbrace{\sum_{i \in I_{j}} h_{i}+\lambda}_{H_{j}}) w_{j}^{2}\right]+\gamma T
$$  
where $l_{j}=\left\{i \mid q\left(x_{i}\right)=j\right\}$  is set of training example indices landing in leaf $j$.  
- Simpliﬁes to  
$$
\sum_{j=1}^{T}\left[G_{j} w_{j}+\frac{1}{2}\left(H_{j}+\lambda\right) w_{j}^{2}\right]+\gamma T
$$  
- For ﬁxed $q(x)$ (i.e. ﬁxed tree partitioning), objective minimized when leaf node values are   
$$
w_{j}^{*}=-G_{j} /\left(H_{j}+\lambda\right)
$$  
- Plugging $w^∗_j$ back in, this objective reduces to  
$$
-\frac{1}{2} \sum_{j=1}^{T} \frac{G_{j}^{2}}{H_{j}+\lambda}+\gamma T
$$  
- which we can think of as the loss for tree partitioning function $q(x)$.  
- If time were no issue, we could search over all trees to mininize this objective.  
- Expression to evaluate a tree’s node assignment function $q(x)$:  
$$
-\frac{1}{2} \sum_{j=1}^{T} \frac{G_{j}^{2}}{H_{j}+\lambda}+\gamma T
$$  
where $G_{j}=\sum_{i \in I_{j}} g_{i}$  for examples $i$ assigned to leaf node $j$,and $H_{j}=\sum_{i \in l_{j}} h_{i}$  
- Suppose we’re considering splitting some data into two nodes: $L$ and $R$.  
- Loss of tree with this one split is  
$$
-\frac{1}{2}\left[\frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}\right]+2 \gamma
$$  
-Without the split – i.e. a tree with a single leaf node, loss is
$$
-\frac{1}{2}\left[\frac{\left(G_{L}+G_{R}\right)^{2}}{H_{L}+H_{R}+\lambda}\right]+\gamma
$$  

## XGBoost: Node Splitting Criterion
- We can deﬁne the gain of a split to be the reduction in objective between tree with and without split:  
$$
\text { Gain }=\frac{1}{2}\left[\frac{G_{L}^{2}}{H_{L}+\lambda}+\frac{G_{R}^{2}}{H_{R}+\lambda}-\frac{\left(G_{L}+G_{R}\right)^{2}}{H_{L}+H_{R}+\lambda}\right]-\gamma
$$  
- Tree building method: recursively choose split that maximizes the gain.






